In [1]:
import rebound
import numpy as np

from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [2]:
sim = rebound.Simulation()
particle_names = ["Sun", "Jupiter", "Saturn", "Uranus", "Neptune"]
# we use the NASA horizon database to look up the Sun and planets
sim.add(particle_names)

# let's give all the particles a unique hash (based on its name)
for i, particle in enumerate(sim.particles):
    particle.hash = particle_names[i]

sim.status()

Searching NASA Horizons for 'Sun'... Found: Sun (10).
Searching NASA Horizons for 'Jupiter'... Found: Jupiter Barycenter (5).
Searching NASA Horizons for 'Saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'Uranus'... Found: Uranus Barycenter (7).
Searching NASA Horizons for 'Neptune'... Found: Neptune Barycenter (8).
---------------------------------
REBOUND version:     	3.7.2
REBOUND built on:    	Jan 24 2019 14:05:33
Number of particles: 	5
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0009979160528340067 y=0.007485182961707869 z=-5.1155848352918975e-05 vx=-0.0004661327119434601 vy=9.923679283419417e-05 vz=1.201576402163755e-05>
<rebound.Particle object, m=0.0009547919152112404 x=-1.925047844933738 y=-4.9743790436874775 z=0.06369193996338163 vx=0.40383923420405904 vy=-0.13740064070127625 vz=-0.00846347503398179>
<rebound.Particle object,

In [3]:
sim.save("solar_system_outer_planets.bin")

sim = rebound.Simulation.from_file("solar_system_outer_planets.bin")

In [4]:
def simulate_fly_by(sim, intruder, visualize=False):
    intruder.hash = "intruder"
    
    sim.add(intruder)
    
    intruder_distance = np.linalg.norm(sim.particles["intruder"].xyz)
    sim.exit_max_distance = intruder_distance*1.01
    
    while True:
        try:
            sim.integrate(sim.t+5)
            
            if visualize:
                fig = rebound.OrbitPlot(sim,color=True,unitlabel="[AU]")
                display(fig)
                plt.close(fig)
                clear_output(wait=True)

        except rebound.Escape as error:
            #print(error)
            for i, particle in enumerate(sim.particles):
                distance = np.linalg.norm(particle.xyz)
                if distance > sim.exit_max_distance:
                    #print("Removed", i, str(particle.hash))
                    sim.remove(hash=particle.hash)
                    sim.move_to_com()
                    
            return sim


In [5]:
def calc_escape_velocity(sim, particle):
    #sim.move_to_hel()
    
    r = np.linalg.norm(particle.xyz)
    G = sim.G
    m = sim.particles[0].m
    
    return np.sqrt(2 * G * m / r)

In [6]:
def strong_regime(resolution=100, n_trials=50):
    print("Starting strong regime simulation with resolution {}, {} trials each...".format(resolution, n_trials))
    xs = np.linspace(1, 50, resolution)
    f_eject = np.ones(resolution)
    
    for i, x in enumerate(xs):
        print("Running r_min =", x)
        eject_count = 0.
        
        # run n_trials trials detecting ejection directly after fly-by
        for j in range(n_trials):
            # get a fresh simulation
            sim = rebound.Simulation.from_file("solar_system_outer_planets.bin")
            sim = randomize_sim(sim)
            
            intruder = rebound.Particle(m=1.,x=x,y=-1000.,vy=2.)
            
            sim = simulate_fly_by(sim, intruder)
            
            sim.move_to_hel()
            for particle in sim.particles:
                v = np.linalg.norm(particle.vxyz)
                v_esc = calc_escape_velocity(sim, particle)
                if v > v_esc:
                    eject_count += 1
                    break
        print("Detected", eject_count, "ejections out of", n_trials, "trials.")
        f_eject[i] = eject_count / n_trials
        print(f_eject[i])

    
    return (xs, f_eject)      

In [7]:
def calc_mutual_hill_radius(p1, p2, m_host):
    """
    Calculates mutual Hill radius of particle 1 and 2.
    """
    mutual_hill_radius = (p1.a + p2.a) / 2. * np.cbrt((p1.m + p2.m) / (3. * m_host))
    return mutual_hill_radius
    

In [8]:
def orbit_list(simulation, period, particle, step_size):
    """
    Creates list of points on an orbit.
    """
    locations = []
    total_time = 0
#     Temporary simulation, adding sun and the particle we want the orbit from
    temp_sim = rebound.Simulation()
    temp_sim.add(simulation.particles[0])
    temp_sim.add(particle)
    particle = temp_sim.particles[1]
#     Integrating over exactly one orbit
    while total_time < period:
        temp_sim.integrate(temp_sim.t+step_size)
        total_time += step_size
        locations.append(particle.xyz)
        
    return np.array(locations)


In [ ]:
def check_orbit_crossing(simulation):
    """
    Checks in a simulation whether any orbits cross.
    """
    
#     Creating and saving lists with points on orbits
    locationslist = []
    for i, particle in enumerate(simulation.particles[1:]):
        orbit = particle.calculate_orbit()
        step_size = orbit.P * orbit.rhill / (2 * np.pi * orbit.a)
        locationslist.append(orbit_list(simulation, 
                                        orbit.P, particle, step_size))

#     creating distance matrix
    for i, loc1 in enumerate(locationslist):
        for j, loc2 in enumerate(locationslist[i+1:]): 
            dist_mat = spatial.distance_matrix(loc1, loc2)
            if dist_mat[np.where(dist_mat < mutual_rhill(simulation.particles[i+1], 
                                                         simulation.particles[j+i+2]))].size > 0:
                print(f"Planet {i+1} and {i+j+2} (counting from star) will collide!")
                return True
            
    return False

In [ ]:
#sim.status()

orbit = particle.calculate_orbit()
print(orbit)

Let's calculate the AMD, and put all of the AMD in the orbits of any pair, and check whether the orbits will cross, in that case: (plausible) unstable. Or: if not: AMD-stable.

In [ ]:
def check_amd_stability(simulation):
    """
    Checks in a simulation the total AMD, 
    and puts this total AMD in orbits of any pair,
    and checks if the orbits would cross.
    """
    
    # extracting mass, reduced mass, eccentricity and inclination of each planet:
    
    sum_AMD = 0
    
    for i, particle in enumerate(simulation.particles[1:]):
        orbit = particle.calculate_orbit()
        
        
        
    

Let's define a function to predict the stabillity of a system directly after a fly-by.

Instabillity can be defined in a number of ways. The simplest being direct ejection from the system.

This function will try to analyze the stability of a system based on direct observations of it's properties.

In [ ]:
def analyze_stability(sim):
    
    if check_immediate_ejection(sim) == True:
        return False
    
    elif check_orbit_crossing(sim) == True:
        return False
    
    elif check_kozai(sim) == True:
        return False
    
    elif check_AMD(sim) == True:
        return False
    
    else:
        return True
    
    
    

In [ ]:
def check_immediate_ejection(sim):
    # move to Sun frame
    sim.move_to_hel()
    
    # calculate velocity of each particle and compare to escpae velocity
    for particle in sim.particles:
        v = np.linalg.norm(particle.vxyz)
        v_esc = calc_escape_velocity(sim, particle)
        if v >= v_esc:
            return True
    
    return False
    

In [ ]:
#xs, f_eject = strong_regime(resolution=30, n_trials=5000)

#plt.plot(xs, f_eject)

In [ ]:
def randomize_sim(sim):
    sim.integrate(np.random.random()*10**3)
    return sim

In [ ]:
sim = rebound.Simulation.from_file("solar_system_outer_planets.bin")
x=36.
intruder = rebound.Particle(m=1.,x=x,y=-1000.,vy=2.)
simulate_fly_by(sim, intruder, visualize=True)